In [1]:
import mosek
print("MOSEK is successfully installed!")


MOSEK is successfully installed!


In [2]:
import cvxpy as cp

x = cp.Variable()
prob = cp.Problem(cp.Minimize(x**2), [x >= 1])
result = prob.solve(solver=cp.MOSEK)
print("Optimal value:", result)


Optimal value: 1.0000000137609035


In [23]:
import numpy as np

phi_uy = np.array([[9, 0],[0, 8]])

phi_ux = np.array([[2, 5, 0, 0],[0, 0, 9, 1]])
phi_xx = np.array([[1, 4, 0, 0],[2, 0, 0, 0], [0,0,9,6],[0,0,1,5]])
phi_xy = np.array([[2, 0],[1.5, 0],[0, 1],[0, 6]])

phi_xx_inv = np.linalg.inv(phi_xx)

K = phi_uy - phi_ux @ phi_xx_inv @ phi_xy
print(phi_uy)
print(K)

[[9 0]
 [0 8]]
[[ 5.9375      0.        ]
 [ 0.         13.79487179]]


In [24]:
zero_matrix = np.zeros_like(phi_xx)
phi_xx_extend = np.block([[phi_xx, zero_matrix],[phi_xx, phi_xx]])
zero_matrix = np.zeros_like(phi_uy)
phi_uy_extend = np.block([[phi_uy, zero_matrix],[phi_uy, phi_uy]])
zero_matrix = np.zeros_like(phi_ux)
phi_ux_extend = np.block([[phi_ux, zero_matrix],[phi_ux, phi_ux]])
zero_matrix = np.zeros_like(phi_xy)
phi_xy_extend = np.block([[phi_xy, zero_matrix],[phi_xy, phi_xy]])


phi_xx_extend_inv = np.linalg.inv(phi_xx_extend)

K_extend = phi_uy_extend - phi_ux_extend @ phi_xx_extend_inv @ phi_xy_extend
print(K_extend)

[[ 5.9375      0.          0.          0.        ]
 [ 0.         13.79487179  0.          0.        ]
 [ 5.9375      0.          5.9375      0.        ]
 [ 0.         13.79487179  0.         13.79487179]]
